#  Module 3: Defining Agents (LLM Agents)

In this module, we’ll **define an agent** using ADK’s **LLM Agent** (often aliased as `Agent`). You’ll learn the required fields, optional controls, how to scaffold the app with the **CLI**, and how to run it locally.

**What you’ll do**
1. Understand the LLM Agent shape (\*latest docs)
2. Scaffold a project with `adk create`
3. Inspect the **folder structure** created by ADK
4. Implement a minimal `LlmAgent` in `agent.py`
5. Run locally via **CLI** (`adk run`, `adk web`) and via **Runner** (Python)

**References**: ADK Agents and LLM Agents (identity, instruction, tools, schemas), ADK CLI Reference (create/run/web/api_server), Built-in tools note, and Gemini key env vars. :contentReference[oaicite:0]{index=0}

## 0) Concept: What is an LLM Agent?
An **LLM Agent** (class: `LlmAgent`) is the "thinking" unit: it uses a model (e.g., `gemini-2.0-flash`) to reason, follow **instructions**, and optionally call **tools**. (It’s distinct from **Workflow Agents** which orchestrate fixed flows.) :contentReference[oaicite:1]{index=1}

**Minimum you must define**:
- `model`: LLM name (e.g., `gemini-2.0-flash`)
- `name`: unique identifier for routing in multi-agent systems
- (Recommended) `description`: short routing summary
- `instruction`: how to behave (task, constraints, output format, when to use tools)
- Optional: `tools`, `generate_content_config`, `input_schema`, `output_schema`, `output_key`, `include_contents`, `planner` (planning), `code execution` feature toggles. :contentReference[oaicite:2]{index=2}

## 1) Scaffold the app with the CLI
Use the official `adk` CLI to **create** and **run** your agent locally. :contentReference[oaicite:3]{index=3}

```bash
# 1) Create a new ADK app from a template in the current folder
adk create my_first_agent

# 2) Run it in an interactive terminal UI (points at the agent folder)
adk run my_first_agent

# 3) Start the local web UI over a directory of agents (each subfolder is one agent)
adk web .

# 4) Alternative: start an API server (FastAPI) for agents
adk api_server .
```

> **Tip**: `adk --version` / `adk --help` to verify install and list commands. :contentReference[oaicite:4]{index=4}

### What structure do I get?
The CLI sets up a folder where **each agent** lives in its own subdirectory with **`__init__.py`** and **`agent.py`**. When you point `adk web` / `adk api_server` to a parent directory, ADK expects **each subfolder to be a single agent** with those files present. :contentReference[oaicite:5]{index=5}

Example (illustrative):
```
my_first_agent/
  __init__.py        # exposes a module named `agent`
  agent.py           # defines the root LLM Agent instance
  requirements.txt   # (optional) extra deps
  .env               # (optional) GOOGLE_API_KEY / GEMINI_API_KEY
```

> Ensure your **Gemini API key** is exported as `GOOGLE_API_KEY` (preferred) or `GEMINI_API_KEY`. If both are set, `GOOGLE_API_KEY` wins. :contentReference[oaicite:6]{index=6}

https://google.github.io/adk-docs/get-started/quickstart/#project-structure

## 2) Implement a minimal LLM Agent (in `agent.py`)
Below is a compact example that:
1) Defines a function **tool** (`get_capital_city`)  
2) Creates an `LlmAgent` with **model**, **name**, **description**, **instruction**, and **tools**  
3) Shows a **Runner** pattern to execute the agent locally in Python (alternative to `adk run`)

*This aligns with the latest LLM Agent doc sections on identity, instruction, tools, and Runner usage.* :contentReference[oaicite:7]{index=7}

In [ ]:
# --- agent.py (example) ---
from typing import Dict
from google.adk.agents import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

# 1) Define a simple function tool
def get_capital_city(country: str) -> str:
    """Return the capital for a given country (toy lookup)."""
    capitals = {"france": "Paris", "japan": "Tokyo", "canada": "Ottawa"}
    return capitals.get(country.lower(), f"Sorry, I don't know the capital of {country}.")

# 2) Define the LLM Agent (identity + behavior + tools)
agent = LlmAgent(
    model="gemini-2.0-flash",              # choose your model
    name="capital_agent",                  # unique identifier
    description="Answers questions about capital cities.",
    instruction=(
        "You provide country capitals.\n"
        "When asked for a capital, call get_capital_city(country).\n"
        "Respond concisely."
    ),
    tools=[get_capital_city],              # pass function directly
)



> **Why this shape?**
- `model`, `name`, `description` → **identity & routing**
- `instruction` → **behavior** (task, constraints, output format, guidance on tools)
- `tools=[...]` → give the agent **capabilities** beyond the base LLM
- You can also add: `generate_content_config` (temperature/tokens), `input_schema`/`output_schema` (structured IO), `output_key` (store result in session state), `include_contents`, or a `planner`. :contentReference[oaicite:8]{index=8}

## 3) Running your agent
### A) CLI (recommended for demos)
```bash
adk run my_first_agent
```
Runs an interactive prompt for the agent in `my_first_agent/`. :contentReference[oaicite:9]{index=9}

### B) Web UI
```bash
adk web .
```
Starts a local server with a web UI for any agent subfolders in the current directory. Each subfolder must contain `__init__.py` and `agent.py`. :contentReference[oaicite:10]{index=10}

### C) Python Runner (programmatic)
Use the `Runner` with an in-memory session service (see code above) to integrate the agent into scripts/tests. :contentReference[oaicite:11]{index=11}